<a href="https://colab.research.google.com/github/aryankotru/MINERVA/blob/min_beta/MINERVA_Preprocessing_Unit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pywt
import csv
import os
from scipy.stats import entropy, kurtosis, skew
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import butter, sosfilt, sosfreqz, lfilter, iirnotch, spectrogram

#TODO:
#make files OOP
#work on interfacing GUI and model(const updating one file and reading from it) and model and game
#select latest file in dir and read that
#more features
#remove indexes and epoch numbers
#add level of decomp back
#test with GUI
#motor imagery data
#variable epoch length data?? maybe fix epoch length to 3 s
#delete data after read/write op

In [110]:
def read_input_file(input_file):
    metadata = {}
    data = []
    with open(input_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('%Sample Rate'):
                key_value = line[1:].strip().split(' = ')
                value = key_value[1]
                fs = int(value.strip().split(' ')[0])

            elif line.startswith('Sample Index'):
                header = line.split(', ')
            else:
                data.append(line.split(', ')) #typecast as int

    return metadata, header[:9], data

def convert_and_write_output(output_file, metadata, header, data):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write metadata
        #writer.writerow(['% Metadata'])
        """for key, value in metadata.items():
          if key == 'Sample Rate':
            fs = int(value.strip().split(' ')[0])"""
            #print(fs)
            #writer.writerow([f'% {key} = {value}'])

        # Write data header
        #writer.writerow(['% Data'])
        """update_header = []
        for header_val in header:
          if header_val.startswith('EXG Channel'):
            header_val = header_val[:].split('EXG Channel ')
            update_header.append(int(header_val[1]))
        writer.writerow(update_header[:])"""

        # Write data rows
        data = [sublist[1:][:8] for sublist in data] #removes index and keeps only 8 channel data from OpenBCI format
        writer.writerows(data)
    return fs

def main(input_file, output_file):
    metadata, header, data = read_input_file(input_file)
    fs = convert_and_write_output(output_file, metadata, header, data)

if __name__ == '__main__':
    input_file = 'OpenBCI-RAW-2024-06-10_14-01-02.txt'  # Replace with your input file path
    output_file = 'output.csv'  # Replace with your desired output file path
    main(input_file, output_file)

1000


In [103]:
metadata, header, data = read_input_file(input_file)
data = [sublist[1:][:9] for sublist in data]
print(data[:1])

[['-73704.02797508574', '-58111.83102271927', '-81013.54015034916', '-76251.83627031282', '-70539.40093987', '-69930.20414474061', '-72061.39827506522', '-105869.75286838447', '0.256']]


global variable declaration

In [22]:
epoch = 710 #calc epoch len from fs and epoch time len
num_epochs = 12
# Sample rate and desired cutoff frequencies (in Hz)*
fs = 250
lowcut = 1
highcut = 30
filter_order = 9
notch_quality_factor = 0.693
wavelet = 'sym9'

freq_bands = {"delta": [0.3, 4], "theta": [4, 8], "alpha": [8, 13], "beta": [13, 30], "gamma": [30, 50]}
chars = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "*", "#"]
targets = [1 , 2 ,3 , 4, 5, 6,7,8,9,10,11,12]
feature_names = ['Mean', 'Standard Deviation', 'Skewness', 'Entropy', 'Energy', 'Kurtosis', 'Median' ,'Variance']
channel_names = ["POz", "PO3", "PO4", "PO5", "PO6", "Oz", "O1", "O2"]
channel_numbers = [1,2,3,4,5,6,7,8]

T =  2.84
nsamples = T * fs
t = np.arange(0, epoch) / fs


input data

In [27]:
#Load EEG data from the csv file
df = pd.read_csv("S001.csv", header = None)
df = np.transpose(df)
df.columns = channel_names
df

,POz,PO3,PO4,PO5,PO6,Oz,O1,O2
0,-52325.520058,-18745.512888,-21403.269255,-15633.971431,-20347.417614,-9890.825146,-21657.900313,-18416.808153
1,-53157.228420,-18924.997385,-22218.034996,-15684.843998,-20429.269697,-9916.037912,-21772.430645,-18462.495116
2,-55446.940985,-18735.454603,-23016.528667,-15365.258775,-20164.356838,-9598.106718,-21580.205654,-18162.221799
3,-55945.653078,-18482.030540,-22753.001616,-15137.583920,-19940.481778,-9375.259839,-21344.081840,-17942.995902
4,-54100.159055,-18461.198715,-21738.858327,-15286.223011,-20024.747850,-9530.604454,-21378.190600,-18058.777932
...,...,...,...,...,...,...,...,...
170395,12259.662882,11984.289407,11000.365675,11004.657209,6387.859964,13515.428514,14307.306070,13128.832765
170396,12163.639794,11957.243798,10855.884007,10989.636838,6245.568767,13490.215748,14325.992127,13112.739510
170397,12169.183026,11944.637414,10865.539960,10982.797204,6247.759238,13501.033992,14342.487713,13133.794852
170398,12247.369423,11972.308872,10988.519251,10995.582401,6361.574314,13502.017468,14304.489750,13105.452842


testing, validation and training data

In [ ]:
train, valid, test = np.split(df, [int(0.6*len((df))), int(0.8*len(df))])
num_rows, num_cols = df.shape
num_epochs = int((num_rows)/epoch)
#num_epochs

data filtering

In [14]:
def notch_filter(data, rate, freq, quality):
      x = scipy.signal.filtfilt(*scipy.signal.iirnotch(freq / (rate / 2), quality), data)
      #https://neuraldatascience.io/7-eeg/erp_filtering.html
      return x

def butter_bandpass(lowcut, highcut, fs, order=filter_order):
        nyq = 0.5 * fs #
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=filter_order):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

data normalization

In [29]:
def dataset_scaler(dataframe, column_names):
  scaler = MinMaxScaler() #normalizing data points, object of class StandardScaler
  minmax_scaled_data = scaler.fit_transform((dataframe))
  np.reshape(minmax_scaled_data, dataframe.shape)
  scaled_data = np.vstack((minmax_scaled_data))
  plt.figure()
  dfa = pd.DataFrame(minmax_scaled_data)
  #dfa.columns = cols #index for columns
  dfa.columns = column_names #index for columns
  return dfa

In [ ]:
#train = dataset_scaler(train)
#df = dataset_scaler(df) #only transform training data

<Figure size 640x480 with 0 Axes>

In [36]:
def calc_entropy(coeffs):
  p_data = coeffs.value_counts()           # counts occurrence of each value
  p_data = data.value_counts()/sum(coeffs.value_counts().values)
  entro = entropy(p_data)  # get entropy from counts
  return entro

def calculate_features(coeffs):
    mean_val = np.mean(coeffs)
    std_val = np.std(coeffs) #maybe try variance
    skewness_val = skew(coeffs)
    entropy_val = calc_entropy(np.abs(coeffs))
    energy = np.sum(coeffs**2)
    kurtosis_value = kurtosis(coeffs)
    median_value = np.median(coeffs)
    variance_value = np.var(coeffs)
    return mean_val, std_val, skewness_val, entropy_val, energy, kurtosis_value, median_value, variance_value

wavelet transform & feature extraction

In [11]:
def feature_generator(df, channel_names, epoch_len):
    num_epochs = int(len(df) / epoch_len)
    feature_names = ['Mean', 'Standard Deviation', 'Skewness', 'Shannon Entropy']

    all_features = []

    for n in range(num_epochs):
        for channel in channel_names:
            data = df[channel].iloc[n * epoch_len:(n + 1) * epoch_len]
            coeffs = pywt.wavedec(data, wavelet)
            for i, coeff in enumerate(coeffs[2:], 1):
                #print(coeff)
                features = calculate_features(coeff)
                all_features.append([channel] + list(features))

    feature_df = pd.DataFrame(all_features, columns=['Channel'] + feature_names)
    feature_df.set_index(['Channel'], inplace=True)

    return feature_df

#result_df = feature_generator(df, channel_names, 710)



In [21]:
def alpha_feature_generator(df, channel_names, epoch_len):
    num_epochs = int(len(df) / epoch_len)
    all_features = []

    for n in range(num_epochs):
        for channel in channel_numbers:
            data = df.iloc[n * epoch_len:(n + 1) * epoch_len, channel-1]
            coeffs = pywt.wavedec(data, wavelet, level = 5)
            """Frequency Bands:
              The frequency range corresponding to each level is calculated. For a sampling frequency of 250 Hz, the levels correspond roughly to:
              Level 1: 62.5 - 125 Hz
              Level 2: 31.25 - 62.5 Hz
              Level 3: 15.625 - 31.25 Hz
              Level 4: 7.8125 - 15.625 Hz
              Approximation: 0 - 7.8125 Hz
              The alpha band (8-13 Hz) lies within Level 4."""
            for i, coeff in enumerate(coeffs[3:], 1):
                #print(i)
                #print("hello")
                features = calculate_features(coeff)
                all_features.append([channel] + list(features))
            features = calculate_features(coeffs[0])
            all_features.append([channel] + list(features))

    feature_df = pd.DataFrame(all_features, columns=['Channel'] + feature_names)
    feature_df.set_index('Channel', inplace=True)

    return feature_df

alpha_feature_generator(df, channel_names, 710)



,Mean,Standard Deviation,Skewness,Entropy,Energy,Kurtosis,Median,Variance
Channel,,,,,,,,
1,0.224386,573.524812,-0.042858,2.813633,3.387987e+07,13.906322,-0.724647,3.289307e+05
1,-17.738357,2525.144576,-0.004107,5.073808,1.211567e+09,-1.437631,198.913287,6.376355e+06
1,-1.855421,592.974496,0.125079,5.694999,1.276389e+08,-1.194922,75.344314,3.516188e+05
1,-305361.409357,561.778058,-0.011762,3.637584,3.543344e+12,-0.476865,-305299.356215,3.155946e+05
2,-0.384686,35.464670,0.300528,4.057491,1.295628e+05,3.710733,-0.419743,1.257743e+03
...,...,...,...,...,...,...,...,...
7,81160.790809,141.067560,0.030779,3.637585,2.503096e+11,-1.496419,81159.402278,1.990006e+04
8,0.277736,12.726419,0.317832,4.416346,1.669001e+04,-0.424133,-1.577589,1.619618e+02
8,-1.381777,20.989787,-0.073892,4.988593,8.407129e+04,-0.571040,-1.674305,4.405711e+02


dataset labeller

In [34]:
level_map = [1,2,3,4,5]
feature_map = [1,2,3,4]
chars = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

#num = 20 for label, num = 1 for feature,  num = 4 for level
#n = 0 for label, n = 1 for feature, n = 2 for level, in order

def drop_column(dataframe, column_name):
  try:
    df = dataframe.drop(columns=column_name, axis =1)
    return df
  except Exception as e:
    return dataframe

def dataset_labeller(dataframe, num_features_per_epoch, chars, n, label):
  num_samples = len(dataframe) // num_features_per_epoch  # 4800 / 20 = 240 samples
  num_labels = len(chars)  # 12 labels

  # Create a list of labels for each 20-row segment
  labels = []
  for i in range(num_samples):
      labels.extend([chars[i % num_labels]] * num_features_per_epoch)
  # Add the labels to the DataFrame
  dataframe.insert(dataframe.shape[1]-n, label , labels, allow_duplicates = False) #df.shape[1] returns the number of columns
  return dataframe

def label_dataset(dataframe, drop_existing : bool):
  try:
    dataframe = dataset_labeller(dataframe, 24, targets, 0, 'Label')
    #dataframe = dataset_labeller(dataframe, 1, feature_map , 1, 'Feature')
    dataframe = dataset_labeller(dataframe, 4 , level_map , 2, 'Level')
    return dataframe
  except ValueError:
    print("One or more columns already exists in dataframe.")
    if drop_existing:
      print("Dropping existing columns.")
      dataframe = drop_column(dataframe, 'Label')
      dataframe = drop_column(dataframe, 'Feature')
      dataframe = drop_column(dataframe, 'Level')
    return dataframe
    pass

main function

In [30]:
def preprocessing(dataframe, epoch_len):

   filtered_df = pd.DataFrame(columns=dataframe.columns)
   num_epochs = int(len(dataframe.index)/epoch_len)
   #print(num_epochs)

   for channel, i in enumerate(dataframe.columns):
      filtered_col_data = []
      for n in range(num_epochs):
          data = dataframe.iloc[n*epoch:(n+1)*epoch, channel]
          data = notch_filter(data, fs, 50, notch_quality_factor)
          data = butter_bandpass_filter(data, lowcut, highcut, fs, order=filter_order)
          filtered_col_data.extend(data)
      filtered_df[i] = filtered_col_data
   #filtered_df = pd.DataFrame(filtered_df, columns = channel_names)
   feature_df = alpha_feature_generator(filtered_df, channel_names, epoch_len) #extract and store features
   filtered_df = dataset_scaler(feature_df, feature_names) #scale dataset AFTER filtering


   #return filtered_df
   return label_dataset(filtered_df, drop_existing = False)
da = preprocessing(df, 710)
da

,Mean,Standard Deviation,Skewness,Entropy,Energy,Kurtosis,Median,Variance,Label
0,0.619983,0.127919,0.007540,0.006856,4.302151e-02,0.278994,0.624299,1.636343e-02,1
1,0.631466,0.027222,0.529324,0.105995,3.592201e-03,0.188490,0.624148,7.410715e-04,1
2,0.630830,0.000273,0.548776,0.296396,6.913454e-07,0.304951,0.624150,7.463950e-08,1
3,0.088446,0.894302,0.520760,0.301961,7.979248e-01,0.014087,0.068259,7.997761e-01,1
4,0.626655,0.045658,0.004668,0.023948,5.481332e-03,0.279927,0.624294,2.084662e-03,1
...,...,...,...,...,...,...,...,...,...
7675,0.777860,0.238194,0.449684,0.302481,5.666309e-02,0.013895,0.768340,5.673649e-02,8
7676,0.633716,0.032121,0.966317,0.024012,2.713014e-03,0.279865,0.624035,1.031827e-03,8
7677,0.630660,0.006769,0.441237,0.119685,2.221199e-04,0.188774,0.624144,4.582350e-05,8
7678,0.630812,0.000068,0.420399,0.308779,4.355347e-08,0.315805,0.624150,4.702164e-09,8


<Figure size 640x480 with 0 Axes>

In [ ]:
def dataset_output(file_path):
  file_names = []

  for x in os.listdir(file_path):
    if x.endswith(".csv"):
      file_names.append(x)
      #print(file_names)
   #file_path = ["S002.csv", "S003.csv", "S004.csv","S005.csv","S006.csv","S007.csv","S008.csv","S009.csv" ,"S010.csv"]
  for file in file_names:
    df = pd.read_csv(file, header = None)
    df = np.transpose(df)
    df.columns = channel_names
    da = preprocessing(df, 710)
    da.to_excel(f'MIN{file_names.index(file)+1}eta.xlsx', sheet_name='sheet1',  float_format = "%.8f")
  return

dataset_output("/content")


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
def dataset_input(file_path, pattern):
  file_names = []
  #file_path = ['MIN1delta.xlsx', 'MIN2delta.xlsx', 'MIN3delta.xlsx', 'MIN4delta.xlsx', 'MIN5delta.xlsx', 'MIN6delta.xlsx', 'MIN7delta.xlsx', 'MIN8delta.xlsx', 'MIN9delta.xlsx', 'MIN10delta.xlsx']
  #for file in file_path:
  for x in os.listdir(file_path):
    if x.endswith(pattern):
      file_names.append(x)
  data = [pd.read_excel(file) for file in file_names]
  df = pd.concat(data, axis = 0)

  return df

data = dataset_input("/content", "zeta.xlsx")

#data.to_excel("MINzeta.xlsx")
#FIND A WAY TO PASS 2D DATA IN ONE GO

NameError: name 'os' is not defined

In [5]:
data = pd.read_excel("MINzeta.xlsx")




,Channel,Epoch,Mean,Standard Deviation,Skewness,Shannon Entropy,Label
0,1,0,-2.944000e-05,0.004536,-0.426979,2.721067,1
1,1,0,-6.700000e-07,0.000046,-0.624637,3.284336,1
2,1,0,2.105727e+00,0.144627,-0.390493,3.635198,1
3,2,0,-1.648000e-05,0.002541,-0.428060,2.924088,1
4,2,0,-3.700000e-07,0.000026,-0.608286,3.527789,1
...,...,...,...,...,...,...,...
57595,7,239,-1.000000e-06,0.000069,-0.625451,3.163346,12
57596,7,239,2.117982e+00,0.220529,-0.344054,3.632054,12
57597,8,239,-3.413000e-05,0.005273,-0.426365,2.619051,12
57598,8,239,-7.700000e-07,0.000053,-0.626424,3.170403,12


In [7]:
df = pd.DataFrame(data)
df = df.drop(columns = "Epoch",axis =1)
df


,Channel,Mean,Standard Deviation,Skewness,Shannon Entropy,Label
0,1,-2.944000e-05,0.004536,-0.426979,2.721067,1
1,1,-6.700000e-07,0.000046,-0.624637,3.284336,1
2,1,2.105727e+00,0.144627,-0.390493,3.635198,1
3,2,-1.648000e-05,0.002541,-0.428060,2.924088,1
4,2,-3.700000e-07,0.000026,-0.608286,3.527789,1
...,...,...,...,...,...,...
57595,7,-1.000000e-06,0.000069,-0.625451,3.163346,12
57596,7,2.117982e+00,0.220529,-0.344054,3.632054,12
57597,8,-3.413000e-05,0.005273,-0.426365,2.619051,12
57598,8,-7.700000e-07,0.000053,-0.626424,3.170403,12


In [8]:
# Reshape DataFrame into one feature vector
feature_vector = df.iloc[:24, :].values.flatten()
feature_vector

array([ 1.00000000e+00, -2.94400000e-05,  4.53561000e-03, -4.26978920e-01,
        2.72106718e+00,  1.00000000e+00,  1.00000000e+00, -6.70000000e-07,
        4.60400000e-05, -6.24636520e-01,  3.28433592e+00,  1.00000000e+00,
        1.00000000e+00,  2.10572736e+00,  1.44627020e-01, -3.90492860e-01,
        3.63519789e+00,  1.00000000e+00,  2.00000000e+00, -1.64800000e-05,
        2.54059000e-03, -4.28059530e-01,  2.92408794e+00,  1.00000000e+00,
        2.00000000e+00, -3.70000000e-07,  2.62100000e-05, -6.08286110e-01,
        3.52778919e+00,  1.00000000e+00,  2.00000000e+00,  2.09472399e+00,
        7.96268900e-02, -4.57820130e-01,  3.63685869e+00,  1.00000000e+00,
        3.00000000e+00, -6.87300000e-05,  1.06079400e-02, -4.26413750e-01,
        2.66668570e+00,  1.00000000e+00,  3.00000000e+00, -1.56000000e-06,
        1.07490000e-04, -6.26573400e-01,  3.22745821e+00,  1.00000000e+00,
        3.00000000e+00,  2.13930814e+00,  3.40711720e-01, -3.63038220e-01,
        3.62438769e+00,  

In [ ]:
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pywt
import os
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import butter, sosfilt, sosfreqz, lfilter, iirnotch, spectrogram

class Preprocessing:

    def __init__(self, file):
        self.epoch = 710
        self.num_epochs = 12
        self.fs = 250
        self.lowcut = 1
        self.highcut = 30
        self.filter_order = 9
        self.notch_quality_factor = 0.693
        self.wavelet = 'sym9'
        self.freq_bands = {"delta": [0.3, 4], "theta": [4, 8], "alpha": [8, 13], "beta": [13, 30], "gamma": [30, 50]}
        self.chars = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "*", "#"]
        self.stim_freqs = {9.25: 0, 11.25 : 0, 13.25 : 0, 9.75 : 0.5 , 11.75 : 0.5 , 13.75 : 0.5, 10.25 : 1, 12.25 : 1, 14.25: 1, 10.75 : 1.5, 12.75 : 1.5, 14.75 : 1.5}
        self.channel_names = ["POz", "PO3", "PO4", "PO5", "PO6", "Oz", "O1", "O2"]
        self.T = 2.84
        self.nsamples = self.T * self.fs
        self.t = np.arange(0, self.epoch) / self.fs

        self.df = self.load_data(file)
        self.train, self.valid, self.test = self.split_data()

    def load_data(self, file):
        df = pd.read_csv(file, header=None)
        df = np.transpose(df)
        df.columns = self.channel_names
        return df

    def split_data(self):
        train, valid, test = np.split(self.df, [int(0.6*len(self.df)), int(0.8*len(self.df))])
        self.num_epochs = int(len(self.df) / self.epoch)
        return train, valid, test

    def notch_filter(self, data):
        x = scipy.signal.filtfilt(*scipy.signal.iirnotch(50 / (self.fs / 2), self.notch_quality_factor), data)
        return x

    def butter_bandpass(self):
        nyq = 0.5 * self.fs
        low = self.lowcut / nyq
        high = self.highcut / nyq
        sos = butter(self.filter_order, [low, high], analog=False, btype='band', output='sos')
        return sos

    def butter_bandpass_filter(self, data):
        sos = self.butter_bandpass()
        y = sosfilt(sos, data)
        return y

    def dataset_scaler(self, dataframe):
        scaler = MinMaxScaler()
        minmax_scaled_data = scaler.fit_transform(dataframe)
        np.reshape(minmax_scaled_data, dataframe.shape)
        scaled_data = np.vstack((minmax_scaled_data))
        dfa = pd.DataFrame(minmax_scaled_data, columns = channel_names)
        return dfa

    def calculate_features(self, coeffs):
        mean_val = np.mean(coeffs)
        std_val = np.std(coeffs)
        skewness_val = scipy.stats.skew(coeffs)
        entropy_val = entropy(np.abs(coeffs))
        return mean_val, std_val, skewness_val, entropy_val

    def feature_generator(self, df):
        channel_list = []
        channel_dict = {}
        num_epochs = int(len(df.index) / self.epoch)

        for channel in range(len(self.channel_names)):
            channel_list = []

            for n in range(num_epochs):
                data = df.iloc[n*self.epoch:(n+1)*self.epoch, channel]
                coeffs = pywt.wavedec(data, self.wavelet)

                for i, coeff in enumerate(coeffs[2:], 1):
                    mean, std, sk, se = self.calculate_features(coeff)
                    channel_list.append(mean)
                    channel_list.append(std)
                    channel_list.append(sk)
                    channel_list.append(se)

                mean, std, sk, se = self.calculate_features(coeffs[0])
                channel_list.append(mean)
                channel_list.append(std)
                channel_list.append(sk)
                channel_list.append(se)

            channel_dict[self.channel_names[channel]] = channel_list
        feature_df = pd.DataFrame.from_dict(channel_dict)
        return feature_df

    def dataset_labeller(self, dataframe, num_features_per_epoch, chars, n, label):
        num_samples = len(dataframe) // num_features_per_epoch
        num_labels = len(chars)

        labels = []
        for i in range(num_samples):
            labels.extend([chars[i % num_labels]] * num_features_per_epoch)

        dataframe.insert(dataframe.shape[1]-n, label, labels, allow_duplicates=False)
        return dataframe

    def label_dataset(self, dataframe):
        try:
            dataframe = self.dataset_labeller(dataframe, 20, self.chars, 0, 'Label')
            dataframe = self.dataset_labeller(dataframe, 1, [1, 2, 3, 4], 1, 'Feature')
            dataframe = self.dataset_labeller(dataframe, 4, [1, 2, 3, 4, 5], 2, 'Level')
            return dataframe
        except ValueError:
            print("One or more columns already exist in dataframe.")
            return dataframe

    def preprocess(self):
        filtered_df = pd.DataFrame(columns=self.df.columns)
        num_epochs = int(len(self.df.index) / self.epoch)

        for channel, i in enumerate(self.df.columns):
            filtered_col_data = []
            for n in range(num_epochs):
                data = self.df.iloc[n*self.epoch:(n+1)*self.epoch, channel]
                data = self.notch_filter(data)
                data = self.butter_bandpass_filter(data)
                filtered_col_data.extend(data)
            filtered_df[i] = filtered_col_data

        filtered_df = self.dataset_scaler(filtered_df)
        feature_df = self.feature_generator(filtered_df)
        return self.label_dataset(feature_df)

if __name__ == "__main__":
    file = "S001.csv"  # Replace with your file path
    preprocessing = Preprocessing(file)
    processed_data = preprocessing.preprocess()
    processed_data.to_excel('MIN10.xlsx', sheet_name='sheet1', index=False, float_format="%.8f")
